In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Load In Data
* [Did Jane Street modify their trading model around day 85?](https://www.kaggle.com/c/jane-street-market-prediction/discussion/201930)

In [ ]:
import pandas as pd
import time

start_time = time.time()

train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
print('Load in data successful!')

print('-----------------------------------------------------------------------')
time_passed_m = int((time.time() - start_time) // 60)
time_passed_s = int((time.time() - start_time) % 60)
print(f'Time usage: {time_passed_m}m. {time_passed_s}s.')

In [ ]:
import numpy as np

SEED = 1111
np.random.seed(SEED)

TRAINING_PGTS = False
MANUAL_VALIDATING = False
TRAINING = False

print(f'TRAINING_PGTS = {TRAINING_PGTS} \n\
MANUAL_VALIDATING = {MANUAL_VALIDATING} \n\
TRAINING = {TRAINING}')

# Preprocessing

In [ ]:
train = train.query('date > 85').reset_index(drop = True)
train = train[train['weight'] != 0]
train.fillna(train.mean(), inplace=True)
train['action'] = ((train['resp'].values) > 0).astype(int)

features = [c for c in train.columns if 'feature' in c]
f_mean = np.mean(train[features[1:]].values, axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
print(X_train.shape, y_train.shape)

if MANUAL_VALIDATING:
  # train: 80%, validate: 20% ( (499-85) * 0.8 + 85 = 416 )
  X_train_v = train[train['date'] <= 416]
  y_train_v = np.stack([(X_train_v[c] > 0).astype('int') for c in resp_cols]).T
  X_train_v = X_train_v.loc[:, features].values

  X_test_v = train[train['date'] > 416]
  y_test_v = np.stack([(X_test_v[c] > 0).astype('int') for c in resp_cols]).T
  X_test_v = X_test_v.loc[:, features].values

  print(X_test_v.shape, y_test_v.shape)

# del train
print('Done data preprocessing!')

# Create Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, BatchNormalization, Dropout, Dense, Activation
# !pip install tensorflow_addons
import tensorflow_addons as tfa


def create_mlp(num_columns, num_labels, hidden_units,
               dropout_rates, label_smoothing, learning_rate):
  
  inp = Input(shape=(num_columns,))
  x = BatchNormalization()(inp)
  x = Dropout(dropout_rates[0])(x)

  for i in range(len(hidden_units)):
    x = Dense(hidden_units[i])(x)
    x = BatchNormalization()(x)
    x = Activation(tf.keras.activations.swish)(x)
    x = Dropout(dropout_rates[i+1])(x)

  x = Dense(num_labels)(x)
  out = Activation('sigmoid')(x)

  model = tf.keras.models.Model(inputs=inp, outputs=out)
  model.compile(
      optimizer = tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
      loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
      metrics = tf.keras.metrics.AUC(name='AUC')
  )

  return model

# Parameters Setting And Model Visualization

In [ ]:
from keras.utils import plot_model

epochs = [200, 40] # PGTSCV folds all stopped bf. 40
batch_size = [4096, 8192]
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

display(plot_model(create_mlp(
    len(features), 5, hidden_units, 
    dropout_rates, label_smoothing, learning_rate
)))

# PurgedGroupTimeSeriesSplit

In [ ]:
#@title

# TODO: make GitHub GIST
# TODO: add as dataset
# TODO: add logging with verbose

import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

* [numpy.linspace](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) Return evenly spaced numbers over a specified interval.

* [numpy.repeat](https://numpy.org/doc/stable/reference/generated/numpy.repeat.html) Repeat elements of an array.

In [ ]:
n_samples = 2000
n_groups = 20
assert n_samples % n_groups == 0

idx = np.linspace(0, n_samples-1, num=n_samples)
X_train_pgts = np.random.random(size=(n_samples, 5))
y_train_pgts = np.random.choice([0, 1], n_samples)
groups = np.repeat(np.linspace(0, n_groups-1, num=n_groups), n_samples/n_groups)
groups.shape

In [ ]:
#@title

from matplotlib.colors import ListedColormap
import numpy as np
import matplotlib.pyplot as plt
    
# this is code slightly modified from the sklearn docs here:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    
    cmap_cv = plt.cm.coolwarm

    jet = plt.cm.get_cmap('jet', 256)
    seq = np.linspace(0, 1, 256)
    _ = np.random.shuffle(seq)   # inplace
    cmap_data = ListedColormap(jet(seq))

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    # Plot the data classes and groups at the end
    ax.scatter(range(len(X)), [ii + 1.5] * len(X),
               c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

    ax.scatter(range(len(X)), [ii + 2.5] * len(X),
               c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['target', 'day']
    ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
fig, ax = plt.subplots()
cv_pgts = PurgedGroupTimeSeriesSplit(
    n_splits = 5,
    max_train_group_size = 7, 
    group_gap = 2,
    max_test_group_size = 3
)
plot_cv_indices(cv_pgts, X_train_pgts, y_train_pgts, groups, ax, 5, lw=20)

## With The Real Competition Data

* [Use Early Stopping to Halt the Training of Neural Networks At the Right Time](https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/) <br>Evaluating a Validation Dataset in Keras
Early stopping requires that a validation dataset is evaluated during training. <br>This can be achieved by specifying the validation dataset to the fit() function when training your model.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm # import tqdm

FOLDS = 5 # 5, 10
models = []

if TRAINING_PGTS:

  gkf = PurgedGroupTimeSeriesSplit(n_splits=FOLDS, group_gap=10) # 10, 20
  splits = list(gkf.split(y_train, groups=train['date'].values))

  for fold, (train_indices, test_indices) in tqdm(enumerate(splits)):
    X_train_pgts, X_test_pgts = X_train.iloc[train_indices, :], X_train.iloc[test_indices, :]
    y_train_pgts, y_test_pgts = y_train[train_indices], y_train[test_indices]
    # 5 folds shape:
      # (144426, 130) (237204, 130)
      # (372534, 130) (263792, 130)
      # (621401, 130) (270184, 130)
      # (889748, 130) (282063, 130)
      # (1175505, 130) (312613, 130)
    
    # model
    tf.keras.backend.clear_session()
    model = create_mlp(len(features), 5, hidden_units,
                      dropout_rates, label_smoothing, learning_rate)

    er = EarlyStopping(patience = 8, 
                      restore_best_weights = True, 
                      monitor = 'val_loss')
    ReduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                                    factor = 0.1,
                                                    patience = 8,
                                                    verbose = 1,
                                                    mode = 'min')
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = f'../input/js-model-v1/model3-1_{SEED}_{fold}.hdf5',
        save_weights_only = True,
        verbose = 0,
        monitor = 'val_loss',
        save_best_only = True)

    nn_callbacks = [er, ReduceLR, model_checkpoint_callback]

    model.fit(X_train_pgts, y_train_pgts,
              validation_data = (X_test_pgts, y_test_pgts),
              epochs = epochs[0],
              batch_size = batch_size[1],
              callbacks = nn_callbacks)
    models.append(model)

else:

  for fold in tqdm(range(FOLDS)):

    tf.keras.backend.clear_session()
    model = create_mlp(len(features), 5, hidden_units,
                       dropout_rates, label_smoothing, learning_rate)
    model.load_weights(f'../input/js-model-v1/model4_{SEED}_{fold}.hdf5')
    models.append(model)

# Manually Validate / Model Load In

* [tf.keras.callbacks.EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)

* [What is ROC_AUC Curve](https://medium.com/analytics-vidhya/what-is-roc-auc-curve-52d71b93fa2f) AUC is Area Under Curve which is the area under the ROC plot. <br>So how AUC is important. whenever you want to compare the performance of two ML models so the one with higher AUC is performing well than the other. <br>As the AUC will only increase if and only if ROC has points on the top left as it conveys that for some threshold the model has high TPR and low FPR.

* [Lookahead bug when loading saved model #1373](https://github.com/tensorflow/addons/issues/1373)

In [ ]:
# callbacks = EarlyStopping(monitor='AUC', patience=20, verbose=2, mode ='auto')
# # EarlyStopping doesn't apply when there is no validation data
# # , and the AUC should be changed to val_(sth).

if MANUAL_VALIDATING:
  er = EarlyStopping(patience = 8, 
                    restore_best_weights = True, 
                    monitor = 'val_loss')
  ReduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                                  factor = 0.1,
                                                  patience = 8,
                                                  verbose = 1,
                                                  mode = 'min')
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath = '../input/js-model-v1/js_model_v4_weights.h5',
      save_weights_only = True,
      verbose = 0,
      monitor = 'val_loss',
      save_best_only = True)

  nn_callbacks = [er, ReduceLR, model_checkpoint_callback]

  clf = create_mlp(len(features), 5, hidden_units, 
                  dropout_rates, label_smoothing, learning_rate)
  history = clf.fit(
      X_train_v, y_train_v, 
      epochs = epochs[0], 
      batch_size = batch_size, 
      verbose = 1, 
      validation_data = (X_test_v, y_test_v), 
      callbacks = nn_callbacks
  )

  del X_train_v, y_train_v, X_test_v, y_test_v

if TRAINING:
  tf.keras.backend.clear_session()
  tf.random.set_seed(SEED)
  clf = create_mlp(len(features), 5, hidden_units, 
                  dropout_rates, label_smoothing, learning_rate)
  clf.fit(X_train, y_train, epochs=epochs[0], # epochs[0] == 200
          batch_size=batch_size[1], verbose=1, 
          # callbacks=nn_callbacks, 
          )

  # clf.save('../input/js-model-v1/js_model_v2.h5')
  clf.save_weights('../input/js-model-v1/js_model_v2-1_weights.h5')
  print('Training succeeded!\n')

else:
  # clf = tf.keras.models.load_model('../input/js-model-v1/js_model_v2.h5')
  clf = create_mlp(len(features), 5, hidden_units,
                  dropout_rates, label_smoothing, learning_rate)
  clf.load_weights('../input/js-model-v1/js_model_v2_weights.h5')

  print('Loading succeeded!\n')

clf.summary()  

models.append(clf)

# Threshold Tuning

In [ ]:
#@title

def utility_score_bincount(date, weight, resp, action):
    count_i = len(np.unique(date))
    # print('weight: ', weight)
    # print('resp: ', resp)
    # print('action: ', action)
    # print('weight * resp * action: ', weight * resp * action)
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

In [ ]:
# use the last 50 date data as the tuning data_____________
valid = train[train['date'] > 450]
valid_np = valid.loc[:, features].values
target_cols = ['action']

# predict_______________________________________________
f = np.median
models = models[-2]

valid_pred = np.zeros((len(valid), len(target_cols)))
valid_pred = models(valid_np, training=False).numpy()
valid_pred = f(valid_pred, axis=1)

# utility_________________________________________________________________________
best_threshold, best_u_score = 0.5, 0
for i in range(4500, 5500):
    thres = float(i) / 10000
    slice_valid_pred = valid_pred.copy()
    slice_valid_pred = np.where(slice_valid_pred >= thres, 1, 0).astype(int)
    valid_u_score = utility_score_bincount(date=valid.date.values, weight=valid.weight.values,
                                           resp=valid.resp.values, action=slice_valid_pred)
    # print(f'thresold={thres:.4f}, valid_u_score={valid_u_score:.4f}')
    
    if valid_u_score >= best_u_score:
        best_u_score = valid_u_score
        best_threshold = thres

print(f'best_threshold={best_threshold:.4f}, best_u_score={best_u_score:.4f}')

# Submit

* [20210204 | Speed up your prediction](https://www.kaggle.com/tocha4/20210204-speed-up-your-prediction) # Adopt this notebook and predict faster.

In [ ]:
#@title

# v2 -> 10607.512
# v2 and f3, f4 -> 8575.954
# v3-2 and f3, f4 -> 8055.567
# v4 and f3, f4 -> 7533.243
# f3, f4 -> 7803.533
# f4 -> 8173.289
# v4 -> 6866.934

# with 0.5:
#     mf4 -> 8446.199
#     m2f4 -> 8444.816
#     f9 -> 5315.052
#     8192 f4 -> 7530.415
#     f4 (4), m4f4 (8)-> 7575.449
#     v2 -> 10514.384
#     v2-2 (8192) -> 8509.970
#     m3-1f9 (8192) -> 6062.890
#     8192 m4f4, m3-1f9 -> 6990.913

## 1. (v2 and mf4)
## 2. (mf4)

# 1. v2
# 2. mf4

In [ ]:
th = best_threshold # 0.502, 0.5
# f = np.median

# models = models[-2:] # :, -4 (timeout error)

import janestreet
env = janestreet.make_env()

# speed up___
test_df_columns = ['weight'] + [f'feature_{i}' for i in range(130)] + ['date']
index_features = [n for n, col in enumerate(test_df_columns) if col in features]

for (test_df, pred_df) in tqdm(env.iter_test()):
  
#   if test_df['weight'].item() > 0:
  if test_df['weight'].values[0] > 0:
    
#     x_tt = test_df.loc[:, features].values
    x_tt = test_df.values[0][index_features].reshape(1, -1)
    
    if np.isnan(x_tt[:, 1:].sum()):
        x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
    
#     pred = np.mean([model(x_tt, training=False).numpy() for model in models], axis=0)
    pred = models(x_tt, training=False).numpy() # one model only
    pred = f(pred)
    
#     pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    pred_df.action = int(pred >= th)
    
  else:
    
#     pred_df.action = 0
    pred_df['action'].values[0] = 0
        
  env.predict(pred_df)

# References

* [OWN Jane Street with Keras NN](https://www.kaggle.com/tarlannazarov/own-jane-street-with-keras-nn)

* [Jane Street with Keras NN overfit](https://www.kaggle.com/code1110/jane-street-with-keras-nn-overfit)

* [【中文思路】Try to use NN baseline](https://www.kaggle.com/chixujohnny/try-to-use-nn-baseline)

* [Purged Time Series CV, XGBoost, Optuna 🔪📆](https://www.kaggle.com/marketneutral/purged-time-series-cv-xgboost-optuna#Time-Series-Cross-Validation)

* [[JaneStreet\] MLP inference (stage3)](https://www.kaggle.com/code1110/janestreet-mlp-inference-stage3)

* [Pytorch Resnet Starter[Training]🔥🔥🔥](https://www.kaggle.com/a763337092/pytorch-resnet-starter-training/comments)

* [Searching Best Threshold🔥🔥🔥](https://www.kaggle.com/a763337092/searching-best-threshold)